In [74]:
import importlib
import multiprocessing
from pathlib import Path
import tempfile

import captum.attr as attr
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization
import numpy as np
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader

from binaryLoader import BinaryDataset, RandomChunkSampler, pad_collate_func
from MalConvGCT_nocat import MalConvGCT

import explainability as e

importlib.reload(e)

<module 'explainability' from '/home/lk3591/Documents/MalConv2/explainability.py'>

In [75]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

In [76]:
model = e.load_classifier(device)
dataset = e.get_dataset()
loader = e.get_loader(dataset, BATCH_SIZE=1)

In [77]:
model.zero_grad()
torch.backends.cudnn.enabled = False # See 1

In [105]:
correct, incorrect = 0, 0
with torch.no_grad():
    for i, (X, y) in enumerate(loader):
        #print(X.shape, y.shape)
        if i == 1000:
            break
        outputs, penultimate_activ, conv_active = model(X)
        _, predicted = torch.max(outputs.data, 1)
        if predicted.item() == y.item():
            correct += 1
        else:
            incorrect += 1

In [107]:
print(f"i: {i}, correct %: {correct / i}, incorrect %: {incorrect / i}")

i: 1000, correct %: 0.141, incorrect %: 0.859


In [79]:
token_reference = TokenReferenceBase(reference_token_idx=256)
lig = LayerIntegratedGradients(lambda x: model(x)[0], model.embd)

In [80]:
def get_an_input(index):
    for i, item in enumerate(loader):
        if i == index:
            X, y = item
            return X.to(device), y.to(device)

In [81]:
#vis_data_record = e.interpret(model, item, lig, reference_indices)
#vis_data_record = visualization.VisualizationDataRecord(*vis_data_record)

In [82]:
def forward_func(x):
    return model(x)[0]
ig = attr.IntegratedGradients(forward_func)
ig

In [83]:
X, y = get_an_input(5)
X.shape, X, y

(torch.Size([1, 50767]),
 tensor([[ 61,  34,  69,  ..., 113, 117,  63]], dtype=torch.int16),
 tensor([1]))

In [84]:
outputs, penultimate_activ, conv_active = model(X)
torch.max(outputs.data, 1)[1], outputs.shape, penultimate_activ.shape, conv_active.shape

(tensor([0]), torch.Size([1, 2]), torch.Size([1, 256]), torch.Size([1, 256]))

In [85]:
baseline = token_reference.generate_reference(X.shape[1], device=device).unsqueeze(0)
baseline.shape, baseline

(torch.Size([1, 50767]), tensor([[256, 256, 256,  ..., 256, 256, 256]]))

In [97]:
attr_ig, delta = ig.attribute(X, target=y, baselines=baseline, return_convergence_delta=True)
attr_ig, delta

(tensor([[-0., -0., -0.,  ..., -0., -0., -0.]]), tensor([3.9075]))